In [ ]:
!pip install gdown


In [ ]:
!gdown --id 1_nT0jgyuY-H6rHEOBcb_mX2QO_L5MCx6 -O my_models.zip
!gdown --id 1J3DpD1Lv8Z1hzWPeSTYz8rA5fuvfzJa1 -O part_2_models.zip
!gdown --id 1x-eMm5B0YtCCYGLpwKuKroyQgP4Q5yqg -O part_3_models.zip

!unzip my_models.zip
!unzip part_2_models.zip
!unzip part_3_models.zip

In [ ]:

!gdown --id 1SCFb88vXrKPUb229JW3D2m26TVq10KMU -O face_age.zip
!unzip face_age.zip

import glob
import os # Make sure to import os
import tensorflow as tf # Make sure to import tensorflow


data_dir = "/content/face_age"

for ds_store_file in glob.glob(f"{data_dir}/**/.DS_Store", recursive=True):
    os.remove(ds_store_file)
class_names = sorted(os.listdir(data_dir))
age_labels = [int(name) for name in class_names]  # Convert subdir names to integers
# lookup table for class index → actual age
age_lookup = tf.constant(age_labels, dtype=tf.int32)
# 3. Map dataset labels to actual ages
data = tf.keras.utils.image_dataset_from_directory(data_dir)
data = data.map(lambda x, y: (x, tf.gather(age_lookup, y)))
# 4. Now apply your age-to-category mapping
def label_to_category(image, label):
    category = tf.where(
        label < 13, 0,
        tf.where(
            label < 20, 1,
            tf.where(label < 60, 2, 3)
        )
    )
    return image, tf.cast(category, tf.int32)

dataset = data.map(label_to_category)
data = data.map(lambda x,y:(x/255,y))
dataset = dataset.map(lambda x,y:(x/255,y))
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size
  # 10% for testing
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

length = dataset.cardinality().numpy()

train_size_d = int(length * 0.7)
val_size_d = int(length * 0.2)
test_size_d = length - train_size_d - val_size_d

train_dataset = dataset.take(train_size_d)
val_dataset = dataset.skip(train_size_d).take(val_size_d)
test_dataset = dataset.skip(train_size_d + val_size_d)

# **Part 1 Regression and Classification(Best Models)**

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
model_1 = load_model('/content/model_1.keras')
model_reg_skip = load_model('/content/model_reg_skip.keras')

### **Classification**

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, Accuracy
def evaluate_classification_model(model, test_dataset):
    pre = Precision()
    re = Recall()
    acc = Accuracy()

    for batch in test_dataset.as_numpy_iterator():
        x, y = batch
        yhat = model.predict(x, verbose=0)

        # Get true and predicted class indices
        y_true = y.argmax(axis=1) if y.ndim > 1 else y
        y_pred = yhat.argmax(axis=1)

        # Update metrics
        pre.update_state(y_true, y_pred)
        re.update_state(y_true, y_pred)
        acc.update_state(y_true, y_pred)

    precision = pre.result().numpy()
    recall = re.result().numpy()
    accuracy = acc.result().numpy()
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score: {f1_score:.4f}")

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "f1_score": f1_score
    }


In [ ]:

model_class = evaluate_classification_model(model_1, test_dataset)


### **Regression**

In [ ]:
def evaluate_models_on_test(models, model_names, test_dataset):
    print(f"{'Model':<20} {'Test Loss (MSE)':<20} {'Test MAE':<15}")
    print("-" * 55)
    for model, name in zip(models, model_names):
        test_loss, test_mae = model.evaluate(test_dataset, verbose=0)
        print(f"{name:<20} {test_loss:<20.4f} {test_mae:<15.4f}")


In [ ]:
evaluate_models_on_test(
    models=[model_reg_skip],  # Use model objects instead of History objects
    model_names=["Model_regression"],
    test_dataset=test
)

# **PART 2 Using Backbone Models and Fine Tuning**


In [ ]:
class Cast(tf.keras.layers.Layer):
    def __init__(self, dtype='float16', **kwargs):
        super(Cast, self).__init__(**kwargs)
        self._dtype = dtype

    def call(self, inputs):
        return tf.cast(inputs, self._dtype)

    def get_config(self):
        config = super().get_config()
        config.update({'dtype': self._dtype})
        return config

    @property
    def dtype(self):
        return self._dtype

    @dtype.setter
    def dtype(self, value):
        self._dtype = value

from tensorflow.keras.utils import register_keras_serializable
register_keras_serializable('custom_layers', 'Cast')(Cast)

age_classifier_autoencoder = load_model('/content/age_classifier_3.keras', custom_objects={'Cast': Cast})
mobilenet = load_model('/content/mobilenet_model.keras')

### **Autoencoder Model**

In [ ]:
model_autoencoder = evaluate_classification_model(age_classifier_autoencoder, test_dataset)

### **Mobile Net model**

In [ ]:
model_mobilenet = evaluate_classification_model(mobilenet, test_dataset)

# **PART 3 Bias in Data**


In [ ]:
one_gender = load_model('/content/gender_model_male.keras')
both_gender = load_model('/content/gender_model_general.keras')

### **Model Trained on 1 Gender(Male)**

In [ ]:
model_1_gender = evaluate_classification_model(one_gender, test_dataset)

### **Model Trained on Both the Genders**

In [ ]:
model_both_gender = evaluate_classification_model(both_gender, test_dataset)